# Climate Risk Analysis: Mongolia Dzud and Summer Drought Analysis 2009-2010

## Introduction

Mongolia is highly vulnerable to climate extremes due to its cold continental climate and strong dependence on pastoral livestock subsistence farming. One of the most severe climate hazards affecting the country is *dzud*, a compound winter disaster of varying types, usually characterised by extreme cold, long term or deep snow cover, and ice crusts that prevent livestock from accessing pasture and damage or make infrastructure unusable. Dzud can lead to major livestock losses, threatening rural subsistence farmers and national level food security, further increasing urban struggles with overcrowding and over worked services.

**Types of dzud**
- **White Dzud**: Deep snow covers the ground, burying grass and infrastructure.
- **Black Dzud**: Little persistent snow combined with poor rainfall in the summer, causing little grass on the pastures.
- **Iron dzud**: Icy impenetrable crust forms over the ground, making grass impossible to access.

The 2009-2010 dzud stands out as one of the worst climate disasters in Mongolia's modern history, resulting in the loss of many millions of liverstock and serious socio economic impacts across the entire country. Importantly, dzud risk deos not arise from the winter alone. A summer of poor pasture growth, typically caused by below average rainfall or drought, can leave livestock in a more precarious position going into winter amplifying the potential extremes of the winter.

This notebook analyses the 2009-2010 dzud as a *compound climate event*, starting in the summer 2009 and its precipitation conditions and finishing in the extreme winter spanning 2009-2010. By analysing both phases together, this analysis aims to show the full climatic pathway through which the risk appeared in Mongolia during this period.

**Objectives**
- Assess the precipitation levels and spread in the 2009 summer as a preconditioning factor in dzud risk using satellite rainfall data.  
- Demonstrate the severity of the winter conditions in 2009 and 2010 using temperature and snow related indicators.  
- Compare how the summer and winter of 2009-2010 conditions are against the historical baseline to determine their extremes.    
- Examine spatial patterns of climate stress in Mongolia and illustrate local conditions using Ulaanbaatar as a representative location.  
- Provide a seasonal and compound perspective on climate risk that later supports a comparison with the drought and extreme rainfall risk in Chad.

## Data Sources

### Primary Sources

### Supplementary Sources

### Data Notes

## Historical Baseline Context

### Why 1981-2009?

### Statistical References

## Summer 2009 Preconditioning Analysis

## Winter 2009-2010 Dzud Severity Analysis

## Localised Summer and Winter Analysis: Ulaanbaatar

## Seasonal Patterns and Compounding Risks

## Summary Statistics

## Conclusion and Next Steps